# Introduction

In this exercise, you will create and submit predictions for a Kaggle competition. You can then improve your model (e.g. by adding features) to apply what you've learned and move up the leaderboard.

Begin by running the code cell below to set up code checking and the filepaths for the dataset.

In [41]:
# Set up code checking
# Set up filepaths
import os
os.chdir(os.path.join(os.path.expanduser('~'), 'kaggle'))

In [42]:
# Import helpful libraries
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [61]:
# Load the data, and separate the target
iowa_file_path = 'data/house-price-data/train.csv'
home_data: pd.DataFrame = pd.read_csv(iowa_file_path)

test_data_path = 'data/house-price-data/test.csv'
test_data: pd.DataFrame = pd.read_csv(test_data_path)

y = home_data['SalePrice']
home_data.drop(columns=['SalePrice'], inplace=True)

numeric_features = [col for col in home_data.columns if is_numeric_dtype(home_data[col].dtype)]
categorical_features = [col for col in home_data.columns if is_string_dtype(home_data[col].dtype)]
# categorical values which appear as numbers
hidden_categorical = ['MSSubClass']
categorical_features.extend(hidden_categorical)
numeric_features = [n for n in numeric_features if (n not in hidden_categorical)]
print("numerical features")
print(numeric_features)
print('categorical features')
print(categorical_features)

for nf in numeric_features:
    home_data[nf].fillna(home_data[nf].mean(), inplace=True)
    test_data[nf].fillna(test_data[nf].mean(), inplace=True)

for cf in categorical_features:
    home_data[cf].fillna(home_data[cf].mode()[0], inplace=True)
    test_data[cf].fillna(test_data[cf].mode()[0], inplace=True)

# onehotencoding for categorical features
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

cat_features_df = pd.DataFrame(ohe.fit_transform(home_data[categorical_features]), columns=ohe.get_feature_names_out())

test_cat_features_df = pd.DataFrame(ohe.transform(test_data[categorical_features]), columns=ohe.get_feature_names_out())

home_data = pd.concat([home_data[numeric_features], cat_features_df], axis=1)
test_data = pd.concat([test_data[numeric_features], test_cat_features_df], axis=1)
print(test_data.shape, home_data.shape)
print(test_data.head())
# print(home_data.head())

numerical features
['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
categorical features
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageF

### Feature selection

In [62]:
features = home_data.columns
print(features)

Index(['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'MSSubClass_60', 'MSSubClass_70', 'MSSubClass_75', 'MSSubClass_80',
       'MSSubClass_85', 'MSSubClass_90', 'MSSubClass_120', 'MSSubClass_160',
       'MSSubClass_180', 'MSSubClass_190'],
      dtype='object', length=303)


In [63]:
# Select columns corresponding to features, and preview the data
X = home_data[features]
# X.head()

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Define a random forest model
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

relative_error = rf_val_mae/y.mean()
print(f"Validation MAE for Random Forest Model: {rf_val_mae: .0f}")
print(f"Relative error: {relative_error}")

Validation MAE for Random Forest Model:  16574
Relative error: 0.09160796648367446


In [64]:

# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor()

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(X, y)

RandomForestRegressor()

Now, read the file of "test" data, and apply your model to make predictions.

In [66]:
# path to file you will use for predictions
# test_data_path = '../input/test.csv'

# # read test data file using pandas
# test_data = pd.read_csv(test_data_path)

# # create test_X which comes from test_data but includes only the columns you used for prediction.
# # The list of columns is stored in a variable called features
test_X = test_data[features]

# # make predictions which we will submit. 
test_preds = rf_model_on_full_data.predict(test_X)
print(test_preds)
print(type(test_preds), len(test_preds))

[128295.5  154728.75 180405.32 ... 152244.99 113393.   229756.53]
<class 'numpy.ndarray'> 1459


Before submitting, run a check to make sure your `test_preds` have the right format.

# Generate a submission

Run the code cell below to generate a CSV file with your predictions that you can use to submit to the competition.

In [67]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('code/house-prices-competition/submission_1.csv', index=False)